# 1. 데이터 수집 및 전처리

KOSPI200 종목 리스트·영업일 보정·수익률 수집 파이프라인입니다. 프로젝트 루트를 `sys.path`에 추가한 뒤 `src` 모듈을 사용합니다.

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))

import pandas as pd
from pykrx import stock
from datetime import datetime, timedelta
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## 영업일 보정 함수

휴장일인 경우 다음/이전 영업일을 반환하는 헬퍼 함수입니다.

In [ ]:
def get_next_trading_day(date):
    """date: 'YYYYMMDD' 문자열. 휴장일이면 다음 영업일 반환."""
    while True:
        df = stock.get_market_ohlcv_by_date(date, date, "005930")
        if not df.empty:
            return date
        dt = datetime.strptime(date, "%Y%m%d") + timedelta(days=1)
        date = dt.strftime("%Y%m%d")

def get_previous_trading_day(date):
    """date: 'YYYYMMDD' 문자열. 휴장일이면 이전 영업일 반환."""
    while True:
        df = stock.get_market_ohlcv_by_date(date, date, "005930")
        if not df.empty:
            return date
        dt = datetime.strptime(date, "%Y%m%d") - timedelta(days=1)
        date = dt.strftime("%Y%m%d")

## KOSPI200 종목 리스트 및 수익률 수집

`Kospi200DataLoader`로 기간 내 KOSPI200 편입 종목 리스트와 연도별 수익률을 수집합니다.

In [ ]:
from src.kospi200_loader import Kospi200DataLoader

loader = Kospi200DataLoader(start_year=2015, end_year=2024)
code_list = loader.kospi200_list()
codes_df = pd.DataFrame({'주식코드': code_list})
print(f"KOSPI200 종목 수: {len(code_list)}")

# 연도별 수익률 수집 (선택)
# rtn_df = loader.kospi200_rtn(codes_df)
# rtn_df.to_csv(PROJECT_ROOT / "data" / "kospi200_returns.csv", index=False)